# Mount to Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
#images_path = "/content/drive/MyDrive/Traffic Sign Detection/ALL DATA"
images_path = "/data/ALL Data"
import sys
#sys.path.append('/content/drive/MyDrive/Traffic Sign Detection/source')
sys.path.append('/source')
import models as md
import utils
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset Preparation
Below code is to resize images and create test and training folders. No need to run them again.

In [ ]:
from PIL import Image
import os

# Target size for every image
target_size = (500, 500)
os.makedirs("data/resized500", exist_ok=True)

# Iterating through class_0 images to resize
for image_file in os.listdir("{}/class_0".format(images_path)):
    image_path = os.path.join("{}/class_0".format(images_path), image_file)
    img = Image.open(image_path)
    img_resized = img.resize(target_size)

    #kaydet
    output_path = os.path.join("data/resized500", image_file)
    img_resized.save(output_path)

# Iterating through class_1 images to resize
for image_file in os.listdir("{}/class_1".format(images_path)):
    image_path = os.path.join("{}/class_1".format(images_path), image_file)
    img = Image.open(image_path)
    img_resized = img.resize(target_size)

    #kaydet
    output_path = os.path.join("data/resized500", image_file)
    img_resized.save(output_path)

print("Image resizing complete.")

# Splitting Data Set Into Training and Test

In [ ]:
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
images_path = "/content/drive/MyDrive/Traffic Sign Detection/ALL DATA"

# Path to store training and test datasets
destination_folder = "/content/drive/MyDrive/Traffic Sign Detection/cnn1/training and test"

# Class names
classes = ["class_0", "class_1"]

# Create train and test folders
train_folder = os.path.join(destination_folder, "train")
test_folder = os.path.join(destination_folder, "test")

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

In [ ]:
#copy images from original dataset to train and test folders
for class_name in classes:
    class_folder = os.path.join(images_path, class_name)

    # Create class folders in train and test folders
    train_class_folder = os.path.join(train_folder, class_name)
    test_class_folder = os.path.join(test_folder, class_name)

    os.makedirs(train_class_folder, exist_ok=True)
    os.makedirs(test_class_folder, exist_ok=True)


    # Split images into train and test sets
    image_list = os.listdir(class_folder)
    train_images, test_images = train_test_split(image_list, test_size=0.2, random_state=42)

    # Copy images to train folder
    for img in train_images:
        source_path = os.path.join(class_folder, img)
        destination_path = os.path.join(train_class_folder, img)
        shutil.copy(source_path, destination_path)

    # Copy images to test folder
    for img in test_images:
        source_path = os.path.join(class_folder, img)
        destination_path = os.path.join(test_class_folder, img)
        shutil.copy(source_path, destination_path)

print("Dataset split into train and test folders.")

# CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

In [ ]:
# Paths to store splitted datasets
train_data_dir = "/data/cnn1/training and test/train"
test_data_dir = "/data/cnn1/training and test/test"

batch_size = 32

# Create an ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Normalize pixel values to the range [0, 1]
    shear_range=0.2,     # Shear transformations
    zoom_range=0.2,      # Zoom transformations
    horizontal_flip=True  # Horizontal flips
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create train generators with established batch size
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(500, 500),
    batch_size=batch_size,
    class_mode="binary"  # Assumes two classes
)

# Create validation generators with established batch size
validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(500, 500),
    batch_size=batch_size,
    class_mode="binary")

Found 852 images belonging to 2 classes.
Found 272 images belonging to 2 classes.


In [ ]:
model = md.firstcnn()
checkpoint = ModelCheckpoint("best_tsmodel1.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1)
# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks = [checkpoint]
)

# Saving the model
model.save("/savedmodels/best_tsmodel1.h5")

Epoch 1/20
26/26 [==============================] - ETA: 0s - loss: 0.7008 - accuracy: 0.7073
Epoch 1: val_accuracy improved from -inf to 0.71875, saving model to best_tsmodel1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - 121s 4s/step - loss: 0.7008 - accuracy: 0.7073 - val_loss: 0.4808 - val_accuracy: 0.7188
Epoch 2/20
26/26 [==============================] - ETA: 0s - loss: 0.4325 - accuracy: 0.7841
Epoch 2: val_accuracy improved from 0.71875 to 0.84375, saving model to best_tsmodel1.h5
26/26 [==============================] - 119s 5s/step - loss: 0.4325 - accuracy: 0.7841 - val_loss: 0.3775 - val_accuracy: 0.8438
Epoch 3/20
26/26 [==============================] - ETA: 0s - loss: 0.4059 - accuracy: 0.8134
Epoch 3: val_accuracy did not improve from 0.84375
26/26 [==============================] - 92s 4s/step - loss: 0.4059 - accuracy: 0.8134 - val_loss: 0.4670 - val_accuracy: 0.7422
Epoch 4/20
26/26 [==============================] - ETA: 0s - loss: 0.3687 - accuracy: 0.8244
Epoch 4: val_accuracy improved from 0.84375 to 0.85938, saving model to best_tsmodel1.h5
26/26 [==============================] - 119s 5s/step - loss: 0.3687 - accuracy: 0.8244 - val_loss: 

In [ ]:
#loading the model and seeing results for the whole dataset
model = tf.keras.models.load_model("/savedmodels/best_tsmodel1.h5")

confusion = [[0, 0], [0, 0]]

# CLASS_0
counter = [0, 0]
path = "/data/ALL DATA/class_0"

for filename in os.listdir(path):
    #loading and preprocessing the test image
    img_path = os.path.join(path, filename)
    img = load_img(img_path, target_size=(500, 500))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    #prediction
    prediction = model.predict(img_array)

    #printing the predicted class (0 or 1)
    print(f"Image: {filename}, Prediction: {int(round(prediction[0][0]))}")
    if int(round(prediction[0][0])) == 0:
      counter[0] += 1
    elif int(round(prediction[0][0])) == 1:
      counter[1] += 1

confusion[1][1], confusion[0][1] = counter[0], counter[1]

# CLASS_1
counter = [0, 0]
path = "/data/ALL DATA/class_1"
for filename in os.listdir(path):
    #loading and preprocessing the test image
    img_path = os.path.join(path, filename)
    img = load_img(img_path, target_size=(500, 500))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  #normalize pixel values to the range [0, 1]

    #prediction
    prediction = model.predict(img_array)

    #printing the predicted class (0 or 1)
    print(f"Image: {filename}, Prediction: {int(round(prediction[0][0]))}")
    if int(round(prediction[0][0])) == 0:
      counter[0] += 1
    elif int(round(prediction[0][0])) == 1:
      counter[1] += 1
print(counter)


confusion[0][0], confusion[1][0] = counter[1], counter[0]

In [ ]:
# Printing the confusion matrix
for i in confusion:
  print(i)
